# Sales Voice Agent

This notebook provides an interactive voice agent for sales activities with calendar integration and RAG capabilities, powered by Ollama.

## Setup and Imports

In [ ]:
import os
import json
import pyttsx3
import speech_recognition as sr
import faiss
import ollama
from sentence_transformers import SentenceTransformer
import re
import datetime
from calendar_helper import create_event

DOC_PATH = "documents.json"
INDEX_PATH = "vector.index"
EMBEDDINGS_PATH = "embeddings.pkl"
CALENDAR_PATH = "calendar.txt"


## RAG (Retrieval-Augmented Generation) Class

In [ ]:
class RAG:
    def __init__(self):
        self.model = SentenceTransformer("all-MiniLM-L6-v2")
        self.index = None
        self.documents = []

    def build_index(self, docs):
        self.documents = docs
        embeddings = self.model.encode(docs)
        index = faiss.IndexFlatL2(embeddings.shape[1])
        index.add(embeddings)
        faiss.write_index(index, INDEX_PATH)
        with open(EMBEDDINGS_PATH, "wb") as f:
            import pickle
            pickle.dump(docs, f)

    def load_index(self):
        self.index = faiss.read_index(INDEX_PATH)
        with open(EMBEDDINGS_PATH, "rb") as f:
            import pickle
            self.documents = pickle.load(f)

    def retrieve(self, query, top_k=3):
        query_vec = self.model.encode([query])
        D, I = self.index.search(query_vec, top_k)
        return [self.documents[i] for i in I[0]]


## Voice Agent Class (Ollama-powered)

In [ ]:
class VoiceAgent:
    def __init__(self):
        self.rag = RAG()
        self.model_name = "llama3.2:3b"
        self.tts = pyttsx3.init()
        self.listener = sr.Recognizer()

    def listen(self):
        with sr.Microphone() as source:
            print("🎤 Listening...")
            audio = self.listener.listen(source)
        try:
            text = self.listener.recognize_google(audio)
            print(f"👂 You said: {text}")
            return text
        except Exception as e:
            print(f"❌ Could not understand: {e}")
            return ""

    def speak(self, text):
        print(f"🤖 Assistant: {text}")
        self.tts.say(text)
        self.tts.runAndWait()

    def answer(self, query):
        context = "\n".join(self.rag.retrieve(query))
        prompt = f"Context: {context}\n\nQuestion: {query}\nAnswer:"
        
        try:
            response = ollama.chat(model=self.model_name, messages=[
                {
                    'role': 'user',
                    'content': prompt
                }
            ])
            return response['message']['content']
        except Exception as e:
            print(f"Error calling Ollama: {e}")
            return "I'm sorry, I couldn't process your request right now."

    def parse_schedule_request(self, user_input):
        user_input_lower = user_input.lower()
        
        # Default values
        event_title = "Sales Call"
        duration = 30
        start_time = datetime.datetime.now() + datetime.timedelta(hours=1)
        
        # Extract event title
        if "call" in user_input_lower:
            if "demo" in user_input_lower:
                event_title = "Product Demo Call"
            elif "sales" in user_input_lower:
                event_title = "Sales Call"
            elif "meeting" in user_input_lower:
                event_title = "Sales Meeting"
            else:
                event_title = "Call"
        
        # Extract duration
        duration_match = re.search(r'(\d+)\s*(min|minute|minutes|hour|hours)', user_input_lower)
        if duration_match:
            value = int(duration_match.group(1))
            unit = duration_match.group(2)
            if unit in ['hour', 'hours']:
                duration = value * 60
            else:
                duration = value
        
        # Extract time information
        time_patterns = [
            r'tomorrow\s+at\s+(\d{1,2})(?::(\d{2}))?\s*(am|pm)?',
            r'today\s+at\s+(\d{1,2})(?::(\d{2}))?\s*(am|pm)?',
            r'(\d{1,2})(?::(\d{2}))?\s*(am|pm)?',
            r'in\s+(\d+)\s*(hour|hours|minute|minutes)',
        ]
        
        for pattern in time_patterns:
            match = re.search(pattern, user_input_lower)
            if match:
                if 'tomorrow' in pattern:
                    start_time = datetime.datetime.now() + datetime.timedelta(days=1)
                    hour = int(match.group(1))
                    minute = int(match.group(2)) if match.group(2) else 0
                    ampm = match.group(3)
                    if ampm == 'pm' and hour != 12:
                        hour += 12
                    elif ampm == 'am' and hour == 12:
                        hour = 0
                    start_time = start_time.replace(hour=hour, minute=minute)
                    break
                elif 'today' in pattern:
                    hour = int(match.group(1))
                    minute = int(match.group(2)) if match.group(2) else 0
                    ampm = match.group(3)
                    if ampm == 'pm' and hour != 12:
                        hour += 12
                    elif ampm == 'am' and hour == 12:
                        hour = 0
                    start_time = datetime.datetime.now().replace(hour=hour, minute=minute)
                    if start_time < datetime.datetime.now():
                        start_time += datetime.timedelta(days=1)
                    break
                elif 'in' in pattern:
                    value = int(match.group(1))
                    unit = match.group(2)
                    if unit in ['hour', 'hours']:
                        start_time = datetime.datetime.now() + datetime.timedelta(hours=value)
                    else:
                        start_time = datetime.datetime.now() + datetime.timedelta(minutes=value)
                    break
                else:
                    # Just time specified, assume today
                    hour = int(match.group(1))
                    minute = int(match.group(2)) if match.group(2) else 0
                    ampm = match.group(3)
                    if ampm == 'pm' and hour != 12:
                        hour += 12
                    elif ampm == 'am' and hour == 12:
                        hour = 0
                    start_time = datetime.datetime.now().replace(hour=hour, minute=minute)
                    if start_time < datetime.datetime.now():
                        start_time += datetime.timedelta(days=1)
                    break
        
        return event_title, start_time, duration

    def schedule_event(self, user_input):
        try:
            event_title, start_time, duration = self.parse_schedule_request(user_input)
            
            # Create the calendar event
            event_link = create_event(
                summary=event_title,
                start_time=start_time,
                duration_minutes=duration,
                description=f"Automatically scheduled based on: {user_input}"
            )
            
            # Format time for speech
            time_str = start_time.strftime("%I:%M %p on %B %d")
            
            return f"I've scheduled a {duration}-minute {event_title} for {time_str}. The event has been added to your calendar."
            
        except Exception as e:
            print(f"Error creating calendar event: {e}")
            return "I'm sorry, I couldn't create the calendar event. Please try again."

    def add_to_calendar(self, info):
        with open(CALENDAR_PATH, "a") as f:
            f.write(info + "\n")

    def process_input(self, user_input):
        user_input_lower = user_input.lower()
        
        # Check for scheduling keywords
        schedule_keywords = ['schedule', 'book', 'appointment', 'meeting', 'call', 'demo']
        if any(keyword in user_input_lower for keyword in schedule_keywords):
            return self.schedule_event(user_input)
        
        # Use RAG for general questions
        return self.answer(user_input)


## Initialize the Voice Agent (Run this cell once to load the model)

In [ ]:
print("🚀 Initializing Sales Voice Agent with Ollama...")
print("📚 Loading Llama 3.2 model (this should be fast!)...")

# Initialize the voice agent
agent = VoiceAgent()

# Test Ollama connection
try:
    test_response = ollama.chat(model=agent.model_name, messages=[
        {'role': 'user', 'content': 'Hello'}
    ])
    print("✅ Ollama connection successful!")
except Exception as e:
    print(f"❌ Ollama connection failed: {e}")
    print("💡 Make sure Ollama is running: brew services start ollama")

# Load documents if available
if os.path.exists(DOC_PATH):
    with open(DOC_PATH, 'r') as f:
        documents = json.load(f)
    
    # Check if index exists, if not build it
    if not os.path.exists(INDEX_PATH):
        print("🔍 Building document index...")
        agent.rag.build_index(documents)
    else:
        print("📖 Loading existing document index...")
        agent.rag.load_index()
else:
    print("⚠️  No documents.json found. RAG features will be limited.")

print("✅ Voice Agent initialized successfully!")
print("\n🎯 Ready to help with sales activities!")
print("💡 You can now use the following cells to interact with the agent.")


## Voice Interaction Cell (Run this cell to start voice conversation)

In [ ]:
# Voice interaction
print("🎤 Starting voice interaction...")
print("💬 Say something or ask a question...")

user_input = agent.listen()
if user_input:
    response = agent.process_input(user_input)
    agent.speak(response)
else:
    print("❌ No speech detected. Please try again.")


## Text Interaction Cell (Run this cell for text-based interaction)

In [ ]:
# Text interaction
user_input = input("💬 Enter your question or request: ")
if user_input:
    response = agent.process_input(user_input)
    print(f"🤖 Assistant: {response}")
    
    # Optionally speak the response
    speak_response = input("🔊 Would you like me to speak the response? (y/n): ")
    if speak_response.lower() in ['y', 'yes']:
        agent.speak(response)


## Schedule Event Cell (Run this cell to schedule calendar events)

In [ ]:
# Schedule an event
print("📅 Scheduling an event...")
print("💬 Describe the event you want to schedule (e.g., 'Schedule a sales call tomorrow at 2pm for 30 minutes')")

user_input = input("Event description: ")
if user_input:
    response = agent.schedule_event(user_input)
    print(f"🤖 Assistant: {response}")
    
    # Optionally speak the response
    speak_response = input("🔊 Would you like me to speak the response? (y/n): ")
    if speak_response.lower() in ['y', 'yes']:
        agent.speak(response)


## Quick Actions Cell (Run this cell for common sales tasks)

In [ ]:
# Quick actions menu
print("⚡ Quick Actions Menu:")
print("1. Schedule a sales call")
print("2. Schedule a product demo")
print("3. Ask about sales strategies")
print("4. Ask about customer data")
print("5. Custom question")

choice = input("\nSelect an option (1-5): ")

if choice == "1":
    response = agent.schedule_event("Schedule a sales call for tomorrow at 10am for 30 minutes")
elif choice == "2":
    response = agent.schedule_event("Schedule a product demo call for tomorrow at 2pm for 45 minutes")
elif choice == "3":
    response = agent.answer("What are effective sales strategies for closing deals?")
elif choice == "4":
    response = agent.answer("What customer information do we have available?")
elif choice == "5":
    custom_question = input("Enter your question: ")
    response = agent.process_input(custom_question)
else:
    response = "Invalid choice. Please try again."

print(f"\n🤖 Assistant: {response}")

# Optionally speak the response
speak_response = input("🔊 Would you like me to speak the response? (y/n): ")
if speak_response.lower() in ['y', 'yes']:
    agent.speak(response)


## System Status Cell (Run this cell to check system status)

In [ ]:
# System status
print("📊 System Status:")
print(f"✅ Voice Agent: Initialized")
print(f"✅ Ollama Model: {agent.model_name}")
print(f"✅ Text-to-Speech: Available")
print(f"✅ Speech Recognition: Available")

# Check Ollama connection
try:
    test_response = ollama.chat(model=agent.model_name, messages=[
        {'role': 'user', 'content': 'Test'}
    ])
    print(f"✅ Ollama: Connected and working")
except Exception as e:
    print(f"❌ Ollama: Connection failed - {e}")

# Check calendar integration
if os.path.exists('credentials.json'):
    print(f"✅ Google Calendar: Credentials found")
else:
    print(f"⚠️  Google Calendar: No credentials.json found")

# Check documents
if os.path.exists(DOC_PATH):
    with open(DOC_PATH, 'r') as f:
        documents = json.load(f)
    print(f"✅ Documents: {len(documents)} documents loaded")
else:
    print(f"⚠️  Documents: No documents.json found")

# Check contacts
if os.path.exists('cleaned_contacts.json'):
    with open('cleaned_contacts.json', 'r') as f:
        contacts = json.load(f)
    print(f"✅ Contacts: {len(contacts)} contacts available")
else:
    print(f"⚠️  Contacts: No cleaned_contacts.json found")

print("\n🎯 Ready for sales activities!")
